# Segmentacja danych multispektralnych

![satelita](https://hackmd.io/_uploads/r1lA5vkx-l.jpg)

*Obraz wygenerowany przez narzędzie generowania obrazów DeepAI.*

## Wstęp

Ziemia nieustannie się zmienia &mdash; miasta się rozrastają, granice lasów przesuwają, a lodowce topnieją. Aby zrozumieć te procesy, naukowcy i inżynierowie coraz częściej sięgają po dane satelitarne. Dzięki nim możemy obserwować naszą planetę z kosmosu, monitorować stan środowiska i wykrywać zmiany, których często nie widać „gołym okiem”.

Jednym z kluczowych rodzajów takich danych są obrazy multispektralne, czyli zdjęcia wykonywane w wielu różnych zakresach promieniowania elektromagnetycznego. Każde z pasm dostarcza innych informacji o powierzchni Ziemi, a ściślej o tym, w jaki sposób dany materiał pochłania i odbija światło. Przykładowo: w świetle widzialnym dostrzegamy naturalne barwy roślin i gleby, natomiast podczerwień pozwala ocenić kondycję roślinności czy wilgotność gruntu.

Technologia ta umożliwia nieinwazyjne tworzenie map, analizę terenów uprawnych (np. pod kątem żyzności), śledzenie postępu suszy, a także monitorowanie zanieczyszczeń czy skutków katastrof naturalnych. Obrazy multispektralne stanowią dziś jeden z fundamentów nowoczesnej teledetekcji (ang. remote sensing).

Każdy materiał na Ziemi &mdash; woda, piasek, beton czy trawa - odbija promieniowanie w charakterystyczny dla siebie sposób. Ten unikalny wzór nazywamy sygnaturą spektralną. Analizując te sygnatury w różnych pasmach, możemy precyzyjnie zidentyfikować obiekty widoczne na zdjęciu.

W praktyce dane te bywają jednak zakłócone przez czynniki atmosferyczne (chmury, parę wodną, pyły) oraz drobne błędy pomiarowe czujników. Dlatego, zanim obrazy multispektralne posłużą do analizy, muszą zostać poddane odpowiedniemu przetworzeniu i korekcji.

## Zadanie

Twoim zadaniem jest przygotowanie klasy przetwarzającej zbiór danych oraz nauczenie modelu segmentującego obrazy multispektralne (mapy terenu).

Pierwszy etap zadania polega na stworzeniu funkcji, która przetworzy dane wejściowe w sposób maksymalizujący skuteczność modelu (bez użycia etykiet). Aby zwiększyć różnorodność zbioru treningowego i poprawić zdolność generalizacji, możesz zastosować różne techniki augmentacji danych, takie jak obrót, skalowanie czy dodawanie szumu (albo inne metody, które uznasz za stosowne).

Podczas analizy danych postaraj się dobrać **minimalny** zestaw pasm (kanałów), który pozwoli uzyskać wysoką jakość segmentacji. Zastanów się, które kanały niosą najwięcej informacji - przykładowo: czy podczerwień wpływa na detekcję roślinności? Wykorzystanie wszystkich pasm nie zawsze jest konieczne; często lepsze rezultaty daje selekcja tylko tych, które zawierają kluczowe informacje dla rozróżnianych klas. Pamiętaj, że nie musisz ograniczać się do surowych wartości kanałów - w procesie przetwarzania możesz wygenerować zupełnie nowe reprezentacje cech.

Drugim etapem jest stworzenie rozwiązania do segmentacji obrazów multispektralnych, czyli przypisanie każdemu pikselowi jednej z czterech klas: wody, lądu, roślinności lub terenów przemysłowych (zgodnie z poniższym przykładem). W tym celu możesz wykorzystać biblioteki takie jak PyTorch i scikit-learn.

![mapy_segmentacji](https://live.staticflickr.com/65535/54927654414_b325b31a02_c.jpg)

Do dyspozycji masz opatrzony etykietami zbiór treningowy i walidacyjny, na których możesz testować swoje podejście. Ostateczna ocena modelu zostanie przeprowadzona na ukrytym zbiorze testowym. Każdy obraz wejściowy składa się z 12 pasm spektralnych, jednak decyzja o tym, ile i które z nich wykorzystasz, należy do Ciebie.

## Opis danych

Dane podzielono na osobne zbiory:

- **treningowy** - $48$ próbek (obrazów),

- **walidacyjny** - $16$ próbek (obrazów).

Do ostatecznej oceny posłuży zbiór **testowy** składający się z $96$ próbek (obrazów), do którego nie masz dostępu.
Każda próbka to obraz o wymiarach $30 \times 30$ pikseli. Każdy piksel posiada przypisaną etykietę klasy (maskę segmentacji).
Każdy piksel opisany jest przez $12$ kanałów odpowiadających pomiarom odbicia światła w różnych zakresach długości fali.

## Kryterium oceny

Twój wynik zależy od dwóch czynników: jakości segmentacji na ukrytym zbiorze testowym oraz liczby kanałów, które zdecydujesz się wykorzystać na wejściu modelu.

Pierwszy składnik funkcji oceny związany jest z liczbą wykorzystanych kanałów $N\_channels$. Oryginalnie każdy piksel złożony jest z 12 kanałów i za użycie wszystkich kanałów otrzymasz 0 punktów za tę część zadania. Wszystkie rozwiązania stosujące co najwyżej 3 kanały otrzymają komplet punktów za ten składnik funkcji oceny, z kolei rozwiązania używające $\lbrace 4, 5, 6, ..., 11\rbrace$ kanałów zostaną ocenione według kwadratowej funkcji skali:

$$\mathtt{channel\_evaluation} = 
\begin{cases} 
    0 &\quad \text{jeżeli }  N\_channels \geq 12 \\
    100 &\quad \text{jeżeli }  N\_channels \leq 3 \\
    100 \cdot \left( \dfrac{12 - N\_channels}{12 - 3} \right)^2 &\quad \text{w pozostałych przypadkach}.
\end{cases}$$

Przykładowo, za zastosowanie 10 kanałów otrzymasz jedynie 5 punktów za tę część zadania, pomnożone przez współczynnik wynoszący $0.25$.
Drugi składnik funkcji oceny związany jest z jakością segmentacji dla map zastosowanych w zadaniu. Wykorzystamy do tego miarę $\text{IoU}$ (Intersection over Union), która ocenia stosunek ilości prawidłowo sklasyfikowanych pikseli danej klasy w danym obrazie, w odniesieniu do ilości wszystkich pikseli tej klasy występujących w rzeczywistej mapie (*ground truth*) lub predykcji modelu. Finalnie, policzona zostanie średnia po wszystkich klasach oraz obrazach znajdujących się w zbiorze testowym. Dla każdego obrazka liczona będzie średnia tylko po klasach, które pojawiają się w *ground truth*.

$$\text{IoU} = \dfrac{1}{M \cdot N} \cdot \sum\limits_{i=1}^{N} \sum\limits_{j=1}^{M_i} \dfrac{\text{ilość prawidłowo sklasyfikowanych pikseli j-tej klasy w i-tej mapie}}{\text{ilość pikseli j-tej klasy w łącznym rzeczywistym obszarze oraz predykcji modelu dla i-tej mapy }}$$

gdzie $N$ to ilość obrazów w danym zbiorze (np. testowym), zaś $M_i$ jest liczbą klas rzeczywiście występującą w $i$-tej mapie. Jeśli średnie $\text{IoU}$ wyniesie nie więcej niż 0.6, otrzymasz za tę część zadania 0 punktów, a jeżeli co najmniej 0.8, wówczas otrzymasz za tę część zadania pełną pulę punktów.

$$\mathtt{segmentation\_evaluation} = 
\begin{cases} 
    0 &\quad \text{jeżeli }  \text{IoU} \leq 0.6 \\
    100 &\quad \text{jeżeli }  \text{IoU} \geq 0.8 \\
    100 \cdot \dfrac{\text{IoU} - 0.6}{0.8 - 0.6} &\quad \text{w pozostałych przypadkach}
\end{cases}$$

Przykład obliczania $\text{IoU}$ dla jednej z klas (wyrażonej za pomocą niebieskich kwadratów) znajduje się na poniższym obrazku. Wynik końcowy $\text{IoU}$ jest uśredniany po wszystkich klasach obecnych w danym obrazie, a także po wszystkich obrazach w zbiorze.

![IoU](https://live.staticflickr.com/65535/54922482577_1a5222581f_b.jpg)

Ostateczny wynik za zadanie będzie stanowił $25\%$ oceny za ilość użytych kanałów oraz $75\%$ punktów za jakość segmentacji, zgodnie ze wzorem:

$$\text{score} = 0.25 \cdot \mathtt{channel\_evaluation} + 0.75 \cdot \mathtt{segmentation\_evaluation}$$

**UWAGA!** Jeśli $\text{IoU}$ wyniesie mniej niż 0.5, otrzymasz 0 punktów za całe zadanie, niezależnie od użytej ilości kanałów!


## Ograniczenia

- Twoje rozwiązanie będzie testowane na Platformie Konkursowej bez dostępu do Internetu oraz w środowisku z GPU. 
- Trening modelu i ewaluacja Twojego finalnego rozwiązania na Platformie Konkursowej nie mogą trwać dłużej niż 5 minut z użyciem GPU.
- Klasa dokonująca wstępnego przekształcenia danych `YourPreprocessing` nie może w żaden sposób korzystać z etykiet zbiorów danych (może tylko przetwarzać same próbki), a segmentacja ma być wykonywana bezpośrednio w klasie `YourModel`.

## Pliki Zgłoszeniowe

Ten notebook uzupełniony o Twoje rozwiązanie (patrz klasa `YourModel` oraz klasa `YourPreprocessing`), w którym przygotujesz zestaw zawierający przekształcanie danych z potencjalną redukcją i modyfikacją kanałów oraz model dokonujący segmentacji na przetworzonych danych.

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na True.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Liczba punktów, którą zdobędziesz, będzie wyliczona na (tajnym) zbiorze testowym na Platformie Konkursowej na podstawie wyżej wspomnianego wzoru, zaokrąglona do liczby całkowitej. Jeśli Twoje rozwiązanie nie będzie spełniało powyższych kryteriów, nie będzie wykonywać się prawidłowo lub zostanie wykryta próba oszustwa, otrzymasz za zadanie 0 punktów.

## Kod Startowy

W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

FINAL_EVALUATION_MODE = False  # Podczas sprawdzania ustawimy tą flagę na True.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

import os
import random
from tqdm import tqdm

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

# Dodatkowe biblioteki, któe możesz wykorzystać w Twoim rozwiązaniu
import xgboost
import sklearn

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "./data"
TRAIN_DATA_PATH = os.path.join(DATA_DIR, "train.npz")
VALID_DATA_PATH = os.path.join(DATA_DIR, "valid.npz")

assert torch.cuda.is_available(), "Nie znaleziono karty graficznej (GPU)!"

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

seed = 12345

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["PYTHONHASHSEED"] = str(seed)

### Ładowanie Danych
Za pomocą poniższego kodu wczytujemy dane zawierające obrazy multispektralne oraz odpowiadające im maski segmentacji. Dane te będą podstawą do trenowania i walidacji Twojego modelu segmentującego.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
# Komórka zawierająca funkcje pomocnicze do przygotowania danych.

class BaseDataset(Dataset):
    """
    Klasa zbioru danych multispektralnych.
    """
    def __init__(self, data_path: str):
        data = np.load(data_path)
        self.bands = data["bands"]
        self.segmentations = data["segmentations"]

    def __len__(self):
        """Zwraca liczbę próbek w zbiorze danych."""
        return self.bands.shape[0]

    def __getitem__(self, idx):
        """Zwraca próbkę o indeksie idx - jej pasma multispektralne oraz segmentację."""
        bands = self.bands[idx]
        segmentations = self.segmentations[idx]

        bands = torch.from_numpy(bands).float()
        segmentations = torch.from_numpy(segmentations).long()

        return bands, segmentations

def setup_data():
    """
    Pobiera zbiory danych wykorzystane w zadaniu.
    """
    import gdown
    os.makedirs(DATA_DIR, exist_ok=True)

    if not os.path.exists(TRAIN_DATA_PATH):
        url = "https://drive.google.com/uc?id=1Nfv3Kd9W8ypjr8RL1jY3VF3yTKU934lz"
        gdown.download(url, TRAIN_DATA_PATH, fuzzy=True)
    
    if not os.path.exists(VALID_DATA_PATH):
        url = "https://drive.google.com/uc?id=1WUwayYErm4CXI7zHUAmZmi23doIKmDm3"
        gdown.download(url, VALID_DATA_PATH, fuzzy=True)

if not FINAL_EVALUATION_MODE:
    setup_data()

### Kod z Kryterium Oceniającym

Kod, zbliżony do poniższego, będzie używany do oceny rozwiązania na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def calculate_miou(y_true, y_pred):
    """
    Kalkuluje metrykę mIoU (mean Intersection over Union) używaną do oceny modelu.
    Funkcja pomocnicza wykorzystywana przy ocenie rozwiązania.
    """
    assert y_true.shape == y_pred.shape
    assert y_true.device == y_pred.device

    num_classes = 4
    device = y_true.device
    batch_size = y_true.shape[0]
    y_true_flat = y_true.reshape(batch_size, y_true.shape[1] * y_true.shape[2])
    y_pred_flat = y_pred.reshape(batch_size, y_pred.shape[1] * y_pred.shape[2])

    # Końcowa średnia jest liczona tylko dla klas obecnych w danych referencyjnych (ground truth)
    intersections = torch.zeros((batch_size, num_classes), dtype=torch.float32, device=device)
    unions = torch.zeros((batch_size, num_classes), dtype=torch.float32, device=device)
    true_present = torch.zeros((batch_size, num_classes), dtype=torch.bool, device=device)

    for cls in range(num_classes):
        y_true_c = (y_true_flat == cls)
        y_pred_c = (y_pred_flat == cls)
        true_present[:, cls] = torch.any(y_true_c, dim=1)

        intersections[:, cls] = torch.sum(y_true_c & y_pred_c, dim=1).to(torch.float32)
        unions[:, cls] = torch.sum(y_true_c | y_pred_c, dim=1).to(torch.float32)

    # Oblicza liczbę istotnych klas dla każdej próbki
    num_present_classes = torch.sum(true_present, dim=1).to(torch.float32)

    # Mimo że obliczenia są wykonywane dla wszystkich klas, sumujemy tylko te klasy, które faktycznie występują (w danych referencyjnych)
    iou_per_class = torch.nan_to_num(intersections / unions, nan=0.0)
    sum_iou = torch.sum(iou_per_class * true_present, dim=1)
    miou_scores = torch.nan_to_num(sum_iou / num_present_classes, nan=0.0).tolist()
    assert len(miou_scores) == batch_size

    return miou_scores


def calculate_channel_count(dataloader):
    """
    Funkcja pomocnicza wykorzystywana przy ocenie rozwiązania.
    Kalkuluje ilość pasm wykorzystywanych przy treningu i ewaluacji modelu.
    """
    tensors = dataloader.dataset.tensors
    
    if len(tensors) != 2:
        raise ValueError("W zbiorze danych powinny znajdować się tylko etykiety i obrazy (__getitem__ powinien zwracać krotkę o wymiarowości 2)")
    
    x, _ = tensors
    
    # x.shape [dataset size, channels, height, width]
    if x.ndim != 4:
        raise ValueError("Przetworzone dane muszą posiadać 4 wymiary [batch size, channels, height, width]")

    if x.shape[0] < 15: #zbiór walidacyjny jest najmniejszy i ma 15 próbek 
        raise ValueError(f"Pierwszy wymiar jest zarezerwowany dla rozmiaru datasetu, Twój kod nie powinien go zmieniać!")

    if x.shape[2] != 30 or x.shape[3] != 30:
        raise ValueError("Przetworzone dane muszą mieć wymiary 30x30 na 3 i 4 wymiarze (.shape[2] == .shape[3] == 30)")

    channels_count = x.shape[1]
    return channels_count

def transform_dataset(preprocessing, dataset:BaseDataset) -> TensorDataset:
    """
    Przetwarza wskazany zbiór danych i zachowuje go w pamięci komputera (RAM). 
    Wywołuje zaimplementowaną przez uczestnika funkcję .transform w klasie przygotowującej zbiory danych.
    """
    processed_labels = []
    processed_images = []
    
    for raw_image, label in dataset:
        processed_labels.append(label.clone())
        
        transformed_image = preprocessing.transform(raw_image.clone())
        processed_images.append(transformed_image)

    labels_tensor = torch.stack(processed_labels) 
    images_tensor = torch.stack(processed_images)  
    memory_dataset = TensorDataset(images_tensor, labels_tensor)
    return memory_dataset

def evaluate(train_model, preprocessing, data_path: str):
    """
    Główna funkcja oceniająca zadanie. 
    Taka sama funkcja będzie wywołana na Platformie Konkursowej.

    1. Dopasowuje klasę przetwarzania danych na zbiorze treningowym.
    2. Przetwarza wskazany zbiór danych za pomocą dopasowanej klasy.
    3. Ocenia model za pomocą metryki mIoU na wskazanym przetworzonym zbiorze danych.
    4. Ocenia ilość pasm wykorzystanych przy ocenie modelu - sprawdza kształt danych.
    """

    # Wczytuje zbiór treningowy i dopasowuje na nim klasę przygotowywania danych 
    train_ds = BaseDataset(data_path=TRAIN_DATA_PATH)
    preprocessing = preprocessing()
    assert hasattr(preprocessing, "fit"), "Funkcja przygotowująca dane musi implementować metodę .fit"
    preprocessing.fit(train_ds)
    
    # Wczytuje docelowy zbiór danych i przetwarza go za pomocą klasy słuącej do przygotowywania danych
    target_ds = BaseDataset(data_path=data_path)
    assert hasattr(preprocessing, "transform"), "Funkcja przygotowująca dane musi implementować metodę .transform"
    transformed_dataset = transform_dataset(preprocessing=preprocessing, dataset=target_ds)
    dataloader = DataLoader(transformed_dataset, batch_size=8, shuffle=False)

    model = train_model()

    if hasattr(model, "to") and callable(getattr(model, "to", None)):
        model = model.to(DEVICE)

    if hasattr(model, "eval") and callable(getattr(model, "eval", None)):
        model.eval()

    mious = []

    # Ocenia model za pomocą metryki mIoU
    with torch.no_grad():
        
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            y_pred = model(x)
            if hasattr(y_pred, "to") and callable(getattr(y_pred, "to", None)):
                y_pred = y_pred.to(DEVICE)
            if not torch.is_tensor(y_pred):
                y_pred = torch.tensor(y_pred)
            y_pred = torch.argmax(y_pred, dim=1)

            assert y_pred.shape == y.shape
            assert y_pred.max() <= 4 and y_pred.min() >= 0

            miou = calculate_miou(y, y_pred)
            mious.extend(miou)

    # Ocenia ilość pasm wykorzystanych przy ocenie modelu
    channels_count = calculate_channel_count(dataloader=dataloader)
    
    # Wylicza średnią mIoU po wszystkich próbkach
    miou = sum(mious) / len(mious)

    return miou, channels_count
    
def compute_score(miou, channels_count):
    """
    Oblicza wynik za zadanie, kalkuluje ilość ostatecznych punktów za zadanie za pomocą wzoru podanego w treści zadania.
    Taka sama funkcja będzie wywołana na Platformie Konkursowej.
    """
    band_score = max(0, min(100, 100 * ((12 - channels_count) / (12 - 3))**2))
    miou_score = max(0, min(100, 100 * (miou - 0.6) / (0.8 - 0.6)))

    print(f"Liczba kanałów: {channels_count}")
    print(f"mIoU: {miou:.3f} \n")

    print(f"Wynik dot. kanałów: {band_score:.3f}")
    print(f"Wynik dot. mIoU: {miou_score:.3f}")

    if miou_score < 0.5:
        total_score = int(0)
    else:
        total_score = 0.25 * band_score + 0.75 * miou_score
        total_score = int(round(total_score))
    print(f"Estymowana liczba punktów za zadanie: {total_score}")
    return total_score

## Przykładowe Rozwiązanie

Poniżej przedstawiamy uproszczone rozwiązanie oparte o regresję liniową, które może posłużyć jako przykład demonstrujący działanie notatnika jak i punkt wyjścia do stworzenia Twojego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
class BasicPreprocessing():
  """
  Przykładowa implementacja klasy przetwarzania zbioru danych.

  Jest to tylko przykładowe rozwiązanie, które nie redukuje liczby kanałów.
  Z im mniejszej liczby kanałów skorzystasz, tym więcej otrzymasz punktów za tę część zadania.
  """

  def __init__(self):
    self.std_per_channel = None
    
  def fit(self, dataset:BaseDataset):

    # Zbiera wszystkie obrazy do jednej macierzy [N_próbek, 12, 30, 30]
    images = []
    for item in dataset:
        image_tensor, _ = item
        images.append(image_tensor)
    images_tensor = torch.stack(images)  # shape: [N, 12, 30, 30]

    # Oblicza std (odchylenie standardowe) po wymiarach: (0: próbka, 2: H, 3: W)
    self.std_per_channel = images_tensor.std(dim=(0, 2, 3))

    return self

  def transform(self, image_tensor: torch.Tensor) -> torch.Tensor:          
      # Zmienia kształt średniej na [12, 1, 1], aby umożliwić broadcasting względem [12, 30, 30].
      # Pamiętaj, to tylko przykładowe rozwiązanie, samo podzielenie przez std nie jest najbardziej efektywnym rozwiązaniem.
      std_reshaped = self.std_per_channel.view(-1, 1, 1)
      
      return image_tensor / std_reshaped

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(12, 4)
    
    def forward(self, bands):
        # bands.shape [b, c, h, w]
        return self.linear(bands.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

### Trening Przykładowego Modelu

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def train_basic_model():

    epochs = 40
    lr = 0.001
    batch_size = 8
    model = BasicModel()
    model = model.to(DEVICE)

    # Przygotowuje parametry przetwarzania wstępnego danych tylko z użyciem
    # zbioru treningowego.

    raw_train_ds = BaseDataset(data_path=TRAIN_DATA_PATH)
    raw_valid_ds = BaseDataset(data_path=VALID_DATA_PATH)

    preprocessing = BasicPreprocessing()
    preprocessing.fit(raw_train_ds)

    train_ds = transform_dataset(preprocessing=preprocessing, dataset=raw_train_ds)
    valid_ds = transform_dataset(preprocessing=preprocessing, dataset=raw_valid_ds)
    train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_dataloader, total=len(train_dataloader), desc="Training"):
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            y_pred = model(x)
            loss = criterion(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            mious = []
            for x, y in tqdm(valid_dataloader, total=len(valid_dataloader), desc="Validation"):
                x = x.to(DEVICE)
                y = y.to(DEVICE)

                y_pred = model(x)
                loss = criterion(y_pred, y)

                valid_loss += loss.item()

                y_pred = torch.argmax(y_pred, dim=1)
                miou = calculate_miou(y, y_pred)
                mious.extend(miou)
            
            valid_loss = valid_loss / len(valid_dataloader)
            print(f"Epoch {epoch+1} loss: {valid_loss}, mIoU: {sum(mious) / len(mious)}")
            
    return model

### Ewaluacja Przykładowego Rozwiązania

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    miou, channels_count = evaluate(train_basic_model, BasicPreprocessing, VALID_DATA_PATH)
    print("-"*50)
    compute_score(miou, channels_count)

## Twoje rozwiązanie
W tej sekcji należy umieścić Twoje rozwiązanie. Wprowadzaj zmiany wyłącznie tutaj!

In [ ]:
# Nie zmieniaj nazwy klasy
# Ta klasa może jedynie przetwarzać próbki, nie może korzystać z etykiet danych.
# Żadna metoda nie może też wykonywać segmentacji.

class YourPreprocessing():
  def __init__(self):
    pass
    
  def fit(self, dataset: BaseDataset):
    return dataset

  def transform(self, image_tensor: torch.Tensor) -> torch.Tensor:
    return image_tensor

In [ ]:
class YourModel(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, bands):
        # losowe predykcje
        segmentation = torch.randint(0, 4, (bands.shape[0], 1, 30, 30))
        # Wykonuje one hot encoding ponieważ evaluate wykorzystuje torch.argmax
        one_hot = torch.zeros(bands.shape[0], 4, 30, 30)
        one_hot.scatter_(1, segmentation, 1)
        return one_hot

In [ ]:
# Nie zmieniaj nazwy funkcji
def train_your_model():
    return YourModel()

## Ewaluacja

Uruchomienie komórki poniżej pozwoli sprawdzić, ile punktów zdobyłoby twoje rozwiązanie na danych walidacyjnych. Na Platformie Konkursowej Twoje rozwiązanie będzie oceniane na zbiorze testowym.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    miou, channels_count = evaluate(train_your_model, YourPreprocessing, VALID_DATA_PATH)
    print("-"*50)
    compute_score(miou, channels_count)

**Pamiętaj:** Podczas sprawdzania model (funkcja trenująca model) i funkcja przetwarzająca dane zostaną ocenione na zbiorze testowym, nie walidacyjnym!
